## Installing neccessary packages

In [ ]:
pip install google-cloud-bigquery
pip install --upgrade google-auth
pip install pandas_gbq

## Connecting to the cloud using a json file that I downloaded for authentication:

In [ ]:
from google.cloud import bigquery
from google.oauth2 import service_account

credentials = service_account.Credentials.from_service_account_file('C:/Users/elais/CIS4400-Group-3d11c40a3827.json')
project_id = 'cis4400-group'
client = bigquery.Client(credentials=credentials,project=project_id)

print(client)

## Creating the dataset on GBQ:

In [ ]:
dataset_ref = client.dataset('NBA_All_Decade')

dataset_object = bigquery.Dataset(dataset_ref)
gbq_dataset = client.create_dataset(dataset_object)
print('Dataset {} created.'.format(gbq_dataset.dataset_id))

In [ ]:
print(dataset_ref)

## Creating pandas DFs to match our relational model and loading the data to gbq:

In [ ]:
# importing necessary modules
from google.cloud import bigquery
import pandas as pd

In [ ]:
# Reading csv files and creating dataframes to load to my data warehouse
matchesDim = pd.read_csv('NBA_Games_all_Decade - NBA_Games_all_Decade.csv')
matchesDim['Date'] = pd.to_datetime(matchesDim.Date) # Changing date to be datetime type so we can analyze by dates
playerDim = pd.read_csv('Players_3D.csv')
DateDim = pd.DataFrame(matchesDim['Date'])
DateDim['Date'] = pd.to_datetime(DateDim.Date)
DateDim['Month'] = pd.DatetimeIndex(DateDim['Date']).month # adding broken-down date columns for further analysis
DateDim['DayOfMonth'] = pd.DatetimeIndex(DateDim['Date']).day
DateDim['Year'] = pd.DatetimeIndex(DateDim['Date']).year
DateDim['DayOfWeek'] = DateDim['Date'].dt.day_name()

In [ ]:
# Loading all dataframes to my data warehouse as tables
matchesDim.to_gbq('Decade.MatchesDimension', project_id='cis4400-group')
playerDim.to_gbq('NBA_All_DNBA_All_ecade.PlayersDimension', project_id='cis4400-group')
DateDim.to_gbq('NBA_All_Decade.DatesDimension', project_id='cis4400-group')